
# **AlexNet implementation in Pytorch & Keras**:

Based on the paper ImageNet dataset classification with deep convolutional neural networks


---






# PyTorch

In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset


In [2]:
transform = transforms.Compose([
    transforms.Resize(227),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
indices = list(range(1000))
train_set = Subset(train_set, indices)
train_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_set = Subset(test_set, indices)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
class AlexNet(nn.Module):
  def __init__(self):
    super(AlexNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0)
    self.relu1 = nn.ReLU(inplace=True)
    self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2)
    self.conv2 = nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2)
    self.relu2 = nn.ReLU(inplace=True)
    self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2)
    self.conv3 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)
    self.relu3 = nn.ReLU(inplace=True)
    self.conv4 = nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1)
    self.relu4 = nn.ReLU(inplace=True)
    self.conv5 = nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1)
    self.relu5 = nn.ReLU(inplace=True)
    self.maxpool5 = nn.MaxPool2d(kernel_size=3, stride=2)
    self.fc1 = nn.Linear(256 * 6 * 6, 4096)
    self.relu6 = nn.ReLU(inplace=True)
    self.dropout1 = nn.Dropout(p=0.5)
    self.fc2 = nn.Linear(4096, 4096)
    self.relu7 = nn.ReLU(inplace=True)
    self.dropout2 = nn.Dropout(p=0.5)
    self.fc3 = nn.Linear(4096, 1000)
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    x = self.conv1(x)
    x = self.relu1(x)
    x = self.maxpool1(x)
    x = self.conv2(x)
    x = self.relu2(x)
    x = self.maxpool2(x)
    x = self.conv3(x)
    x = self.relu3(x)
    x = self.conv4(x)
    x = self.relu4(x)
    x = self.conv5(x)
    x = self.relu5(x)
    x = self.maxpool5(x)
    x = x.view(x.size(0), 256 * 6 * 6)
    x = self.fc1(x)
    x = self.relu6(x)
    x = self.dropout1(x)
    x = self.fc2(x)
    x = self.relu7(x)
    x = self.dropout2(x)
    x = self.fc3(x)
    x = self.softmax(x)
    return x

In [16]:
model = AlexNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Keras

In [7]:
import tensorflow as tf
from tensorflow import keras

In [8]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
X_train, y_train, X_test, y_test = X_train[:10000], y_train[:10000], X_test[:1000], y_test[:1000]
y_train.resize(y_train.shape[0])
y_test.resize(y_test.shape[0])
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, sep = '\n')

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
(10000, 32, 32, 3)
(10000,)
(1000, 32, 32, 3)
(1000,)


In [9]:
def resize_images(images):
    return tf.image.resize(images, [128, 128])
X_train = resize_images(X_train)
X_test = resize_images(X_test)
print(X_train.shape, X_test.shape, sep = '\n')

(10000, 128, 128, 3)
(1000, 128, 128, 3)


In [10]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(128,128,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')])

alexnet_optimizer = optimizer=tf.optimizers.SGD(learning_rate = 0.001)

model.compile(loss='sparse_categorical_crossentropy', optimizer = alexnet_optimizer,  metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 30, 30, 96)          │          34,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 30, 30, 96)          │             384 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 14, 14, 96)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 14, 14, 256)         │         614,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 14, 14, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 6, 6, 384)           │         885,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 6, 6, 384)           │           1,536 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 6, 6, 384)           │         147,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 6, 6, 384)           │           1,536 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 6, 6, 256)           │          98,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 6, 6, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 2, 2, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4096)                │       4,198,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4096)                │      16,781,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │          40,9

 Total params: 22,807,306 (87.00 MB)

 Trainable params: 22,804,554 (86.99 MB)

 Non-trainable params: 2,752 (10.75 KB)

In [11]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 64)


Epoch 1/64
313/313 ━━━━━━━━━━━━━━━━━━━━ 24s 36ms/step - accuracy: 0.1451 - loss: 2.8009 - val_accuracy: 0.3250 - val_loss: 1.8972
Epoch 2/64
313/313 ━━━━━━━━━━━━━━━━━━━━ 25s 17ms/step - accuracy: 0.2638 - loss: 2.1221 - val_accuracy: 0.3590 - val_loss: 1.7597
Epoch 3/64
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.3074 - loss: 1.9332 - val_accuracy: 0.4120 - val_loss: 1.6353
Epoch 4/64
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.3445 - loss: 1.7860 - val_accuracy: 0.4030 - val_loss: 1.6126
Epoch 5/64
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.3801 - loss: 1.6981 - val_accuracy: 0.4320 - val_loss: 1.5719
Epoch 6/64
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.4121 - loss: 1.6077 - val_accuracy: 0.4660 - val_loss: 1.4591
Epoch 7/64
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.4280 - loss: 1.5335 - val_accuracy: 0.4520 - val_loss: 1.4844
Epoch 8/64
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.4609 - loss: 1.4739 - val